# Query Translation in RAG Systems



![Query Translation Architecture](../resources/RAG/2_RAG_Query_Translation_1.png)

This diagram shows:
1. Query Processing Flow
2. Translation Components
3. Integration with RAG Pipeline
4. Enhanced Retrieval Process


### In this article, I will introduce advanced techniques of RAG: Query Translation.
More specifically, we will learn three Query Translation methods: Re-written, Step-back question, and sub-question, to enhance retrieval and generation effectiveness. 

## Re-written

Re-written involves semantically rewriting the original query, maintaining the core meaning while adjusting the language expression to facilitate handling by knowledge bases or retrieval systems.

We will rewrite the original question into three different forms: Q1, Q2, and Q3, and then retrieve documents relevant to each of these three formulations.

### Application Scenarios:

The original question is linguistically vague or unclearly expressed.
Optimization of retrieval matching through different formulations.
### Example:


####  Original question: "What is the capital of France?" 
#### Re-written: "What is the capital of France?" 
#### Effect: The rewritten question is more specific, making it easier to retrieve high-quality answers.


### Common Methods:

#### Multi-Query: Generating multiple different queries for the same question during retrieval to access a more comprehensive or relevant set of documents.
#### RAG Fusion: A strategy to fuse retrieval results during generation. It focuses on integrating information from multiple retrieved documents to generate high-quality answers.

### Advantages:

#### Enhances the retrieval system's understanding of questions.
#### Improves the relevance of retrieval results.

### Challenges:

Maintaining semantic consistency between the rewritten question and the original question.
Avoiding introducing bias or ambiguity.

# Multi-Query
![Multi-Query](../resources/RAG/2_RAG_Multi-Query.png)

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# LANGCHAIN_TRACING_V2 = os.environ.get("LANGCHAIN_TRACING_V2")
# LANGCHAIN_ENDPOINT = 'https://api.smith.langchain.com'
# LANGCHAIN_API_KEY = os.environ.get("LANGCHAIN_API_KEY")

DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')



In [2]:
CHROMA_PATH = "chroma_rag2"

%load_ext autoreload
%autoreload 2

from dbUtils import DBUtils
dbUtils = DBUtils()
dbUtils.clear_database(CHROMA_PATH)


In [3]:
from pdfImportService import pdf_content_into_documents, format_docs

pdf_content_into_documents("../data/BeigeBook_20250115.pdf", CHROMA_PATH)

USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/easonyin/Project/github/AIJourney/RAG/dbUtils.py:39: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/easonyin/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


find chuck ../data/BeigeBook_20250115.pdf:0
find chuck ../data/BeigeBook_20250115.pdf:1
find chuck ../data/BeigeBook_20250115.pdf:2
find chuck ../data/BeigeBook_20250115.pdf:3
find chuck ../data/BeigeBook_20250115.pdf:4
find chuck ../data/BeigeBook_20250115.pdf:5
find chuck ../data/BeigeBook_20250115.pdf:6
find chuck ../data/BeigeBook_20250115.pdf:7
find chuck ../data/BeigeBook_20250115.pdf:8
find chuck ../data/BeigeBook_20250115.pdf:9
find chuck ../data/BeigeBook_20250115.pdf:10
find chuck ../data/BeigeBook_20250115.pdf:11
find chuck ../data/BeigeBook_20250115.pdf:12
find chuck ../data/BeigeBook_20250115.pdf:13
find chuck ../data/BeigeBook_20250115.pdf:14
find chuck ../data/BeigeBook_20250115.pdf:15
find chuck ../data/BeigeBook_20250115.pdf:16
find chuck ../data/BeigeBook_20250115.pdf:17
find chuck ../data/BeigeBook_20250115.pdf:18
find chuck ../data/BeigeBook_20250115.pdf:19
find chuck ../data/BeigeBook_20250115.pdf:20
find chuck ../data/BeigeBook_20250115.pdf:21
find chuck ../data/B

[Document(metadata={'source': '../data/BeigeBook_20250115.pdf', 'page': 0, 'id': '../data/BeigeBook_20250115.pdf:0:0'}, page_content='The Beige BookSummary of Commentary onCurrent Economic Conditions byFederal Reserve DistrictJanuary 2025\nFEDERAL RESERVE SYSTEM'),
 Document(metadata={'source': '../data/BeigeBook_20250115.pdf', 'page': 1, 'id': '../data/BeigeBook_20250115.pdf:1:0'}, page_content='ContentsAbout This Publication....................................................................................................iiNational Summary...........................................................................................................1Federal Reserve Bank of Boston..................................................................................5Federal Reserve Bank of New York..............................................................................8Federal Reserve Bank of Philadelphia.......................................................................12Federal Res

In [4]:

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek

# Initialize Deepseek LLM

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives 
    | llm 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)


In [5]:
question = "What are the US economic trends in 2024?"

generate_queries.invoke(question)

['1. What are the projected economic indicators for the United States in 2024?  ',
 '2. How is the US economy expected to perform in 2024?  ',
 '3. What are the key factors influencing the US economy in 2024?  ',
 '4. What are the forecasts for US GDP, inflation, and employment in 2024?  ',
 '5. How will US fiscal and monetary policies impact economic trends in 2024?']

In [6]:
from langchain_chroma import Chroma
from dbUtils import DBUtils
import pprint

dbUtils = DBUtils()
# Vector store

def retrieve_with_queries():
   
    retriever = dbUtils.get_chroma(CHROMA_PATH).as_retriever(search_kwargs={"k": 5})
    return retriever

In [7]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    print(documents)
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [doc.page_content for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [doc for doc in unique_docs]

# Retrieve
retriever = retrieve_with_queries()
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

[[Document(metadata={'id': '../data/BeigeBook_20250115.pdf:3:0', 'page': 3, 'source': '../data/BeigeBook_20250115.pdf'}, page_content='mation enables comparison of economic conditions in different parts of the country, which can behelpful for assessing the outlook for the national economy.The Beige Book does not have the type of information I’m lookingfor. What other information is available?The Federal Reserve System conducts a wide array of recurring surveys of businesses, house-holds, and community organizations. A list of statistical releases compiled by the Federal ReserveBoard is available here, links to each of the Federal Reserve Banks are available here, and a sum-mary of the System’s community outreach is available here. In addition, Fed Listens events havebeen held around the country to hear about how monetary policy affects peoples’ daily lives andlivelihoods. The System also relies on a variety of advisory councils—whose members are drawnfrom a wide array of businesses, no

11

In [8]:
import pprint

pprint.pprint(docs)

['The Beige BookSummary of Commentary onCurrent Economic Conditions byFederal '
 'Reserve DistrictJanuary 2025\n'
 'FEDERAL RESERVE SYSTEM',
 'sales. Home inventories inched upward, and elevated mortgage rates and high '
 'home prices con-tinued to challenge affordability. Outlooks were cautiously '
 'optimistic.Commercial real estate activity was stable during the reporting '
 'period. Apartment leasing was sea-sonally slow, and rents were flat to down '
 'as apartment operators remained focused on main-taining occupancy. Office '
 'leasing demand picked up in some markets but remained subduedoverall. '
 'Industrial activity was characterized as solid, and rents rose '
 'modestly.Financial ServicesLoan volume accelerated sharply in December. '
 'Credit tightening continued, but loan pricingdeclined, both at the same pace '
 'as six weeks ago. Loan nonperformance rose but at a slowerpace. In addition '
 'to swift growth in loan demand, bankers reported a sizeable pickup in '
 'generalb

In [10]:
from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, "question": itemgetter("question")} 
    | prompt 
    | llm
    | StrOutputParser()
)

result = final_rag_chain.invoke({"question":question})

pprint.pprint(result)


[[Document(metadata={'id': '../data/BeigeBook_20250115.pdf:3:0', 'page': 3, 'source': '../data/BeigeBook_20250115.pdf'}, page_content='mation enables comparison of economic conditions in different parts of the country, which can behelpful for assessing the outlook for the national economy.The Beige Book does not have the type of information I’m lookingfor. What other information is available?The Federal Reserve System conducts a wide array of recurring surveys of businesses, house-holds, and community organizations. A list of statistical releases compiled by the Federal ReserveBoard is available here, links to each of the Federal Reserve Banks are available here, and a sum-mary of the System’s community outreach is available here. In addition, Fed Listens events havebeen held around the country to hear about how monetary policy affects peoples’ daily lives andlivelihoods. The System also relies on a variety of advisory councils—whose members are drawnfrom a wide array of businesses, no

# RAG Fusion
![Fusion](../resources/RAG/2_RAG_Fusion.png)

In [11]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_rag_fusion 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [12]:
question = "What are the current economic trends?"

generate_queries.invoke(question)

['1. **What are the latest global economic trends in 2023?**  ',
 '2. **How is inflation impacting current economic trends worldwide?**  ',
 '3. **What are the emerging economic trends in technology and sustainability?**  ',
 '4. **How do interest rate changes affect current economic trends?**']

In [13]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = doc.page_content
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (doc, score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})

len(docs)
pprint.pprint(docs[0])

('The Beige BookSummary of Commentary onCurrent Economic Conditions byFederal '
 'Reserve DistrictJanuary 2025\n'
 'FEDERAL RESERVE SYSTEM',
 0.06558258417063283)


In [14]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

result = final_rag_chain.invoke({"question":question})

pprint.pprint(result)

('The current economic trends, as summarized in the Beige Book for January '
 '2025, indicate the following:\n'
 '\n'
 '1. **Overall Economic Activity**: Economic activity increased slightly to '
 'moderately across the twelve Federal Reserve Districts in late November and '
 'December. Consumer spending rose moderately, with strong holiday sales '
 'exceeding expectations. However, construction activity decreased due to high '
 'costs for materials and financing, and manufacturing activity slightly '
 'declined.\n'
 '\n'
 '2. **Labor Markets**: Employment ticked up on balance, with six Districts '
 'reporting a slight increase and six reporting no change. Wage growth picked '
 'up to a moderate pace in most Districts, though some reports indicated '
 'easing wage pressures.\n'
 '\n'
 '3. **Consumer Spending**: Consumer spending and travel and tourism activity '
 'expanded moderately, with strong auto sales and modest increases in home '
 'sales.\n'
 '\n'
 '4. **Real Estate**: Resident

# Decomposition

A sub-question breaks down a complex question into multiple independent sub-questions, with each sub-question handled separately. Finally, the answers to the sub-questions are aggregated to form a complete response.

## Application Scenarios:
### Multi-hop questions (questions that require answers across multiple knowledge points).
### Questions that involve logical operations or conditional constraints.

## Example:
### Original Question:
"Among Nobel Prize winners, who are both scientists and writers?"

### Sub-questions:

#### "Who are the scientists among Nobel Prize winners?"
#### "Who are the writers among Nobel Prize winners?"
#### "Who are both scientists and writers?"
## Outcome:
By answering each sub-question independently and then integrating the results, an accurate final answer can be obtained.

![Decomposition](../resources/RAG/2_RAG_Decomposition.png)

In [15]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""

prompt_decomposition = ChatPromptTemplate.from_template(template)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the current economic trends?"
questions = generate_queries_decomposition.invoke({"question":question})

questions

['1. What are the latest GDP growth rates for major economies worldwide?  ',
 '2. How are inflation rates trending in different regions as of the current year?  ',
 '3. What are the recent changes in global unemployment rates and labor market conditions?']

In [16]:
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [17]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

q_a_pairs = ""
for q in questions:
    print("Question: ",q)
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair
    print("Answer: ",answer)
    print("\n")

pprint.pprint(answer)

Question:  1. What are the latest GDP growth rates for major economies worldwide?  
Answer:  The provided context from the Beige Book does not contain specific information about the latest GDP growth rates for major economies worldwide. The Beige Book primarily focuses on qualitative assessments of economic conditions within the 12 Federal Reserve Districts in the United States, including employment, wages, prices, and sector-specific activities like manufacturing, retail, and real estate.

To find the latest GDP growth rates for major economies worldwide, you would need to consult sources such as the International Monetary Fund (IMF), the World Bank, or national statistical agencies, which regularly publish updated economic data, including GDP growth rates. These sources provide comprehensive and up-to-date information on global economic performance.


Question:  2. How are inflation rates trending in different regions as of the current year?  
Answer:  The provided context from the B

# Answer individually
![Individually](../resources/RAG/2_RAG_Individual.png)

In [18]:
from langchain import hub

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

prompt_rag

/Users/easonyin/miniconda3/lib/python3.12/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [19]:
# Answer each sub-question individually 

from langchain import hub

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.invoke(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

/Users/easonyin/miniconda3/lib/python3.12/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [20]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

result = final_rag_chain.invoke({"context":context, "question":question})

pprint.pprint(result)

('Current economic trends reflect a mix of regional variations and global '
 'influences. Inflation remains a key concern, with trends differing across '
 'regions. For instance, some areas, like the Minneapolis District, report '
 'moderating prices at wholesale and retail levels, while others, such as the '
 'Atlanta District, face rising prices for specific goods due to supply '
 'constraints. Overall, inflation shows signs of easing in certain sectors but '
 'remains a persistent challenge.\n'
 '\n'
 'Global trade developments are also shaping economic trends. Manufacturers '
 'are stockpiling inventories in anticipation of higher tariffs, which has '
 'slightly dampened manufacturing activity. Retail sales, however, have seen '
 'robust growth, driven by holiday spending. Concerns over potential tariff '
 'increases and competition from China are creating uncertainty, particularly '
 'in sectors like construction, real estate, and manufacturing. Elevated '
 'interest rates further

# Step-back Question

When the system realizes that the current question is too specific or difficult to answer directly, it takes a step back and asks a broader, more general question instead. This helps to obtain more extensive contextual information.

### Application Scenarios
- The current query is too specific and cannot directly match content in the knowledge base
- There is insufficient information in the retrieval results to answer the original question

### Example
**Original Question:**
> "What was the exact speech content of Einstein at the 1921 Nobel Prize award ceremony?"

**Step-back Question:**
> "Why did Einstein win the 1921 Nobel Prize?"

**Outcome:**
By answering a more general question, the system indirectly gathers relevant contextual information, which supports answering the more specific question later.

### Advantages
- Improves retrieval recall and contextual coverage
- Suitable for scenarios with sparse data or high answering difficulty

### Challenges
- How to automatically determine when a "step back" is needed
- Ensuring that the broader question remains semantically related to the original question



In [21]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
    "input": "What are the key indicators of the US economy according to the latest Beige Book?",
    "output": "What economic metrics does the latest Beige Book highlight for the US economy?"
    },
    {
        "input": "How has the US economy been performing based on the recent Beige Book report?",
        "output": "What is the current assessment of the US economy according to the Beige Book?"
    },
    {
        "input": "What sectors of the US economy are experiencing growth, as reported in the Beige Book?",
        "output": "Which industries in the US economy are seeing expansion, according to the Beige Book?"
    }
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an economist in US. 
            Your task is to step back and paraphrase a question to a more generic step-back question, 
            which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [22]:
generate_queries_step_back = prompt | llm | StrOutputParser()

generate_queries_step_back.invoke({"question": question})

'What are the prevailing patterns and developments in the economy at present?'

In [23]:
from langchain_core.runnables import RunnableLambda

# Response prompt 
response_prompt_template = """You are an economist. I am going to ask you a question. 
Your response should be comprehensive and not contradicted with the following context if they are relevant. 
Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke({"question": question})

pprint.pprint(result)

('The current economic trends, as outlined in the **Beige Book** for January '
 '2025, reflect a mixed but generally positive outlook across the United '
 'States. Below is a comprehensive summary of the key trends:\n'
 '\n'
 '### **1. Overall Economic Activity**\n'
 '- **Growth**: Economic activity increased slightly to moderately across the '
 'twelve Federal Reserve Districts in late November and December 2024.\n'
 '- **Consumer Spending**: Consumer spending rose moderately, with strong '
 'holiday sales exceeding expectations in most Districts.\n'
 '- **Construction**: Construction activity decreased overall due to high '
 'material and financing costs.\n'
 '- **Manufacturing**: Manufacturing activity declined slightly, with some '
 'Districts noting manufacturers stockpiling inventories in anticipation of '
 'higher tariffs.\n'
 '- **Real Estate**: Residential real estate activity was unchanged, as high '
 'mortgage rates continued to constrain demand. Commercial real estate sales

# HyDE
HyDE is an innovative method for Query Translation, particularly suitable for handling complex questions in RAG (Retrieval-Augmented Generation) systems.

The core idea of HyDE is to generate a Hypothetical Document, transforming an abstract natural language question into a representation that the retrieval system can efficiently process.

### Typical HyDE Workflow

#### 1. Input Question
The user inputs a natural language question, such as:
> "Which country was the first to successfully launch an artificial satellite?"

#### 2. Generate Hypothetical Document
Using a language model (such as GPT) to generate a possible answer, for example:
> "The first country to successfully launch an artificial satellite was the Soviet Union, who launched Sputnik 1 in 1957."

This step provides a hypothetical answer with strong semantic context, even if the generated content may not be entirely accurate.

#### 3. Retrieve Relevant Documents
- Embed the hypothetical document into vector space
- Extract keywords and use it as input for the retrieval system
- Search for the most relevant documents in the knowledge base

#### 4. Generate Final Answer
Combine the retrieved documents with the original question and use a generation model to produce the final answer.

### Process Summary
1. **Generate Hypothetical Document:**
   - Use a generation model to produce a hypothetical answer
   - Serve as a semantic expansion of the input question

2. **Retrieve Relevant Documents:**
   - Use the hypothetical document as input
   - Retrieve related documents through embedding or keyword matching

3. **Integrate Retrieval Results:**
   - Combine original query with retrieved documents
   - Generate final comprehensive answer

In [24]:
# HyDE document genration

template = """Please write a economic paper passage to answer the question
Question: {question}
Passage:"""

prompt_hyde = ChatPromptTemplate.from_template(template)

generate_docs_for_retrieval = (prompt_hyde | llm | StrOutputParser())

# Run

hypothetical_doc = generate_docs_for_retrieval.invoke({"question": question})

pprint.pprint(hypothetical_doc)

('**Current Economic Trends: An Overview**\n'
 '\n'
 'The global economy in 2023 is navigating a complex landscape shaped by a '
 'confluence of factors, including inflationary pressures, geopolitical '
 'tensions, and the lingering effects of the COVID-19 pandemic. One of the '
 'most prominent trends is the persistence of elevated inflation rates across '
 'many advanced and emerging economies. Central banks, particularly the '
 'Federal Reserve, the European Central Bank, and the Bank of England, have '
 'responded with aggressive monetary tightening, raising interest rates to '
 'curb inflation. While these measures have shown some success in moderating '
 'price increases, they have also contributed to slower economic growth and '
 'heightened financial market volatility.\n'
 '\n'
 'Another significant trend is the uneven recovery across regions. Advanced '
 'economies, such as the United States and parts of Europe, have demonstrated '
 'resilience, supported by robust labor marke

In [25]:
# Retrieve

retrieval_chain = generate_docs_for_retrieval | retriever 

retireved_docs = retrieval_chain.invoke({"question": question})

retireved_docs

[Document(metadata={'id': '../data/BeigeBook_20250115.pdf:27:0', 'page': 27, 'source': '../data/BeigeBook_20250115.pdf'}, page_content='Federal Reserve Bank ofAtlantaSummary of Economic ActivityThe Sixth District economy grew modestly since mid-November. Employment remained steady, withmany firms reporting plans to hold employment levels flat in 2025. Wages, input costs and pricesrose modestly, and firms’ pricing power was mixed. Consumer spending and travel and tourismactivity expanded moderately, and auto sales were strong. Home sales increased modestly, andexisting home inventories improved. Demand for transportation services rose slightly. Manufac-turing activity continued to decline. Loan growth was moderate. Energy activity increased, but agri-culture demand declined.Labor MarketsEmployment was little changed in recent weeks, as firms held headcounts steady. Many contactsplan to keep staffing levels flat into 2025, although some expect to reduce headcount slightly,largely through

In [26]:
# RAG

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

result = final_rag_chain.invoke({"context": retireved_docs, "question": question})

pprint.pprint(result)

('The current economic trends, as described in the documents, include the '
 'following:\n'
 '\n'
 '1. **Economic Growth**: Economic activity increased slightly to moderately '
 'across the twelve Federal Reserve Districts in late November and December. '
 'The Sixth District (Atlanta) economy grew modestly, while other districts '
 'like Chicago, St. Louis, Minneapolis, and Kansas City also reported slight '
 'to moderate growth.\n'
 '\n'
 '2. **Employment**: Employment levels were generally steady, with some '
 'districts reporting slight increases. Many firms plan to keep staffing '
 'levels flat into 2025, although some expect slight reductions through '
 'attrition. There were also reports of hiring for growth in certain sectors.\n'
 '\n'
 '3. **Wages**: Wage growth was modest, with expectations for 2025 wage '
 'increases roughly in line with pre-pandemic growth rates. Wage pressures '
 'eased notably among white-collar workers, though some roles, such as '
 'frontline workers an

# Prompt

In [ ]:
lang_detection_prompt_template = """
Please detect if the user's question is in Chinese. 
If it is, translate it into English and return only the translated text, without any additional explanation.
User question: {question}
"""

lang_detection_prompt = ChatPromptTemplate.from_template(lang_detection_prompt_template)

lang_detection_chain = (
    lang_detection_prompt 
    | llm 
    | StrOutputParser()
)

question = "2024 美国经济总结？"

translated_question = lang_detection_chain.invoke({"question": question})

print(translated_question)

Summary of the U.S. Economy in 2024?


In [ ]:
# Retrieve
retrieval_chain = lang_detection_chain | retriever
retireved_docs = retrieval_chain.invoke({"question": question})
retireved_docs

[Document(metadata={'id': '../data/BeigeBook_20250115.pdf:27:0', 'page': 27, 'source': '../data/BeigeBook_20250115.pdf'}, page_content='Federal Reserve Bank ofAtlantaSummary of Economic ActivityThe Sixth District economy grew modestly since mid-November. Employment remained steady, withmany firms reporting plans to hold employment levels flat in 2025. Wages, input costs and pricesrose modestly, and firms’ pricing power was mixed. Consumer spending and travel and tourismactivity expanded moderately, and auto sales were strong. Home sales increased modestly, andexisting home inventories improved. Demand for transportation services rose slightly. Manufac-turing activity continued to decline. Loan growth was moderate. Energy activity increased, but agri-culture demand declined.Labor MarketsEmployment was little changed in recent weeks, as firms held headcounts steady. Many contactsplan to keep staffing levels flat into 2025, although some expect to reduce headcount slightly,largely through

In [ ]:
# RAG

template = """Answer the following question in Chinese based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
final_rag_chain = (
    {
        "context":retrieval_chain,
        "question": lang_detection_chain,
    }
    | prompt
    | llm
    | StrOutputParser()
)
result = final_rag_chain.invoke({"question": question})

In [ ]:
pprint.pprint(result)

('根据提供的文档内容，以下是2024年美国经济的简要总结：\n'
 '\n'
 '1. **经济增长**：第六区经济自11月中旬以来略有增长。\n'
 '2. '
 '**就业市场**：就业保持稳定，许多公司计划在2025年保持就业水平不变，部分公司预计会通过自然减员略微减少员工数量。劳动力供应充足，预计未来几个月将继续保持这一趋势。\n'
 '3. '
 '**工资增长**：工资增长较为温和，大多数公司预计2025年的工资增长将与疫情前的增长率大致相同，或略低于2024年的水平。白领工人的工资通胀有所缓解，但某些职位（如一线工人、工会员工或难以填补的技术职位）的工资压力仍然较高。\n'
 '4. **价格和成本**：投入成本和价格略有上升，保险和劳动力是公司的主要成本担忧。食品和运输成本保持稳定。\n'
 '5. **消费和旅游**：消费者支出和旅游活动适度增长，汽车销售强劲。\n'
 '6. **房地产市场**：房屋销售略有增加，现有房屋库存有所改善。\n'
 '7. **制造业**：制造业活动继续下降。\n'
 '8. **贷款增长**：贷款增长适度。\n'
 '9. **能源和农业**：能源活动增加，但农业需求下降。\n'
 '\n'
 '总体来看，2024年美国经济呈现出温和增长的态势，就业市场稳定，工资增长温和，价格和成本略有上升，消费和旅游活动有所增长，但制造业活动继续下滑。')
